In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:

import tensorflow as tf
from tensorflow import keras
import numpy as np

In [ ]:
class LogisticEndpoint(keras.layers.Layer):
    def __init__(self, name=None):
        super(LogisticEndpoint, self).__init__(name=name)
        self.loss_fn = keras.losses.BinaryCrossentropy(from_logits=True)
        self.accuracy_fn = keras.metrics.BinaryAccuracy()

    def call(self, targets, logits, sample_weights=None):
        # Compute the training-time loss value and add it
        # to the layer using `self.add_loss()`.
        loss = self.loss_fn(targets, logits, sample_weights)
        self.add_loss(loss)

        # Log accuracy as a metric and add it
        # to the layer using `self.add_metric()`.
        acc = self.accuracy_fn(targets, logits, sample_weights)
        self.add_metric(acc, name="accuracy")

        # Return the inference-time prediction tensor (for `.predict()`).
        return tf.nn.softmax(logits)

layer = LogisticEndpoint()

targets = tf.ones((2, 2))
logits = tf.ones((2, 2))
y = layer(targets, logits)

print("layer.metrics:", layer.metrics)
print("current accuracy value:", float(layer.metrics[0].result()))
inputs = keras.Input(shape=(3,), name="inputs")
targets = keras.Input(shape=(10,), name="targets")
logits = keras.layers.Dense(10)(inputs)
predictions = LogisticEndpoint(name="predictions")(logits, targets)

model = keras.Model(inputs=[inputs, targets], outputs=predictions)
model.compile(optimizer="adam")

data = {
    "inputs": np.random.random((3, 3)),
    "targets": np.random.random((3, 10)),
}
model.fit(data)

In [ ]:
from keras.layers import Input
from keras.layers.merge import concatenate
from keras.layers import Dense, Dropout, Flatten, Activation, Conv2D
from keras.layers.convolutional import MaxPooling2D, AveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model

In [ ]:
def conv_block(x, nb_filter, nb_row, nb_col, padding = "same", strides = (1, 1), use_bias = False):
    '''Defining a Convolution block that will be used throughout the network.'''
    
    x = Conv2D(nb_filter, (nb_row, nb_col), strides = strides, padding = padding, use_bias = use_bias)(x)
    x = BatchNormalization(axis = -1, momentum = 0.9997, scale = False)(x)
    x = Activation("relu")(x)
    
    return x

In [ ]:
def stem(input):
    '''The stem of the pure Inception-v4 and Inception-ResNet-v2 networks. This is input part of those networks.'''
    
    # Input shape is 299 * 299 * 3 (Tensorflow dimension ordering)
    x = conv_block(input, 32, 3, 3, strides = (2, 2), padding = "same") # 149 * 149 * 32
    x = conv_block(x, 32, 3, 3, padding = "same") # 147 * 147 * 32
    x = conv_block(x, 64, 3, 3) # 147 * 147 * 64

    x1 = MaxPooling2D((3, 3), strides = (2, 2), padding = "same")(x)
    x2 = conv_block(x, 96, 3, 3, strides = (2, 2), padding = "same")

    x = concatenate([x1, x2], axis = -1) # 73 * 73 * 160

    x1 = conv_block(x, 64, 1, 1)
    x1 = conv_block(x1, 96, 3, 3, padding = "same")

    x2 = conv_block(x, 64, 1, 1)
    x2 = conv_block(x2, 64, 1, 7)
    x2 = conv_block(x2, 64, 7, 1)
    x2 = conv_block(x2, 96, 3, 3, padding = "same")

    x = concatenate([x1, x2], axis = -1) # 71 * 71 * 192

    x1 = conv_block(x, 192, 3, 3, strides = (2, 2), padding = "same")
    
    x2 = MaxPooling2D((3, 3), strides = (2, 2), padding = "same")(x)

    x = concatenate([x1, x2], axis = -1) # 35 * 35 * 384
    
    return x


In [ ]:

def inception_A(input):
    '''Architecture of Inception_A block which is a 35 * 35 grid module.'''
    
    a1 = AveragePooling2D((3, 3), strides = (1, 1), padding = "same")(input)
    a1 = conv_block(a1, 96, 1, 1)
    
    a2 = conv_block(input, 96, 1, 1)
    
    a3 = conv_block(input, 64, 1, 1)
    a3 = conv_block(a3, 96, 3, 3)
    
    a4 = conv_block(input, 64, 1, 1)
    a4 = conv_block(a4, 96, 3, 3)
    a4 = conv_block(a4, 96, 3, 3)
    
    merged = concatenate([a1, a2, a3, a4], axis = -1)
    
    return merged

In [ ]:
def inception_B(input):
    '''Architecture of Inception_B block which is a 17 * 17 grid module.'''
    
    b1 = AveragePooling2D((3, 3), strides = (1, 1), padding = "same")(input)
    b1 = conv_block(b1, 128, 1, 1)
    
    b2 = conv_block(input, 384, 1, 1)
    
    b3 = conv_block(input, 192, 1, 1)
    b3 = conv_block(b3, 224, 1, 7)
    b3 = conv_block(b3, 256, 7, 1)
    
    b4 = conv_block(input, 192, 1, 1)
    b4 = conv_block(b4, 192, 7, 1)
    b4 = conv_block(b4, 224, 1, 7)
    b4 = conv_block(b4, 224, 7, 1)
    b4 = conv_block(b4, 256, 1, 7)
    
    merged = concatenate([b1, b2, b3, b4], axis = -1)
    
    return merged

In [ ]:
def inception_C(input):
    '''Architecture of Inception_C block which is a 8 * 8 grid module.'''
    
    c1 = AveragePooling2D((3, 3), strides = (1, 1), padding = "same")(input)
    c1 = conv_block(c1, 256, 1, 1)
    
    c2 = conv_block(input, 256, 1, 1)

    c3 = conv_block(input, 384, 1, 1)
    c31 = conv_block(c2, 256, 1, 3)
    c32 = conv_block(c2, 256, 3, 1)
    c3 = concatenate([c31, c32], axis = -1)

    c4 = conv_block(input, 384, 1, 1)
    c4 = conv_block(c3, 448, 3, 1)
    c4 = conv_block(c3, 512, 1, 3)
    c41 = conv_block(c3, 256, 1, 3)
    c42 = conv_block(c3, 256, 3, 1)
    c4 = concatenate([c41, c42], axis = -1)
  
    merged = concatenate([c1, c2, c3, c4], axis = -1)
    
    return merged

In [ ]:
def reduction_A(input, k = 192, l = 224, m = 256, n = 384):
    '''Architecture of a 35 * 35 to 17 * 17 Reduction_A block.'''

    ra1 = MaxPooling2D((3, 3), strides = (2, 2), padding = "same")(input)
    
    ra2 = conv_block(input, n, 3, 3, strides = (2, 2), padding = "same")

    ra3 = conv_block(input, k, 1, 1)
    ra3 = conv_block(ra3, l, 3, 3)
    ra3 = conv_block(ra3, m, 3, 3, strides = (2, 2), padding = "same")

    merged = concatenate([ra1, ra2, ra3], axis = -1)
    
    return merged


In [ ]:
def reduction_B(input):
    '''Architecture of a 17 * 17 to 8 * 8 Reduction_B block.'''
    
    rb1 = MaxPooling2D((3, 3), strides = (2, 2), padding = "same")(input)
    
    rb2 = conv_block(input, 192, 1, 1)
    rb2 = conv_block(rb2, 192, 3, 3, strides = (2, 2), padding = "same")
    
    rb3 = conv_block(input, 256, 1, 1)
    rb3 = conv_block(rb3, 256, 1, 7)
    rb3 = conv_block(rb3, 320, 7, 1)
    rb3 = conv_block(rb3, 320, 3, 3, strides = (2, 2), padding = "same")
    
    merged = concatenate([rb1, rb2, rb3], axis = -1)
    
    return merged

In [ ]:
def inception_v4(nb_classes = 2, load_weights = True):
    '''Creates the Inception_v4 network.'''
    
    init = Input((299, 299, 3)) # Channels last, as using Tensorflow backend with Tensorflow image dimension ordering
    
    # Input shape is 299 * 299 * 3
    x = stem(init) # Output: 35 * 35 * 384
    
    # 4 x Inception A
    for i in range(4):
        x = inception_A(x)
        # Output: 35 * 35 * 384
        
    # Reduction A
    x = reduction_A(x, k = 192, l = 224, m = 256, n = 384) # Output: 17 * 17 * 1024

    # 7 x Inception B
    for i in range(7):
        x = inception_B(x)
        # Output: 17 * 17 * 1024
        
    # Reduction B
    x = reduction_B(x) # Output: 8 * 8 * 1536

    # 3 x Inception C
    for i in range(3):
        x = inception_C(x) 
        # Output: 8 * 8 * 1536
        
    # Average Pooling
    x = AveragePooling2D((8, 8))(x) # Output: 1536

    # Dropout
    x = Dropout(0.2)(x) # Keep dropout 0.2 as mentioned in the paper
    x = Flatten()(x) # Output: 1536

    # Output layer
    output = Dense(units = nb_classes, activation = "softmax")(x) # Output: 1000

    model = Model(init, output, name = "Inception-v4")   
        
    return model

In [ ]:
if __name__ == "__main__":
    inception_v4 = inception_v4()
    inception_v4.summary()

In [ ]:
inception_v4.compile(
loss=keras.losses.binary_crossentropy,optimizer='adam',metrics=['accuracy'])

In [ ]:
from os import listdir
from matplotlib import image
from skimage.transform import resize

def load_data(path): 
    train_files = []
    for foldername1 in listdir(path):
        filepath1 = path  + "/" + foldername1
        for filename1 in listdir(filepath1):
            train_files.append(filepath1 + "/" + filename1)
            
    # Original Dimensions
    image_width = 299
    image_height = 299
    channels = 3
    
    loaded_images = np.ndarray(shape=(len(train_files), image_height, image_width, channels),dtype=np.float32)
    print(type(loaded_images))
    loaded_class = []
    i = 0
    for foldername in listdir(path):
        filepath = path  + "/" + foldername
        print("Folder : ",filepath)
        for filename in listdir(filepath):
            # load image
            img_data = image.imread(filepath + "/" + filename)
            
            # store loaded image
            img_data = resize(img_data, (299, 299, 3))
            loaded_images[i] = img_data
            loaded_class.append(foldername)
            i = i + 1
            #print('> loaded %s %s' % (filename, img_data.shape))
        print('Loaded: ',i , ' images from ',filepath)
            
    return loaded_images,loaded_class

In [17]:
path = r"/content/drive/MyDrive/Colab Notebooks/Project"
x,y = load_data(path)


<class 'numpy.ndarray'>
Folder :  /content/drive/MyDrive/Colab Notebooks/Project/Covid
Loaded:  563  images from  /content/drive/MyDrive/Colab Notebooks/Project/Covid
Folder :  /content/drive/MyDrive/Colab Notebooks/Project/Normal
Loaded:  929  images from  /content/drive/MyDrive/Colab Notebooks/Project/Normal
Folder :  /content/drive/MyDrive/Colab Notebooks/Project/.ipynb_checkpoints
Loaded:  929  images from  /content/drive/MyDrive/Colab Notebooks/Project/.ipynb_checkpoints


In [18]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.3)

In [19]:
print(train_x.shape)
print(test_x.shape)


(650, 299, 299, 3)
(279, 299, 299, 3)


In [20]:
from keras.utils import to_categorical

import pandas as pd
y_train_values, unique = pd.factorize(train_y)
print('y_train ', y_train_values, unique)


y_test_values, unique = pd.factorize(test_y)
print('y_test ', y_test_values, unique)

y_train_one_hot = to_categorical(y_train_values)
y_test_one_hot = to_categorical(y_test_values)

y_train  [0 1 0 1 0 0 0 0 0 1 1 1 1 1 0 1 0 0 0 1 1 1 0 0 1 1 0 0 0 0 0 1 0 0 1 1 0
 0 0 1 1 1 1 0 0 1 1 0 0 1 0 0 1 0 1 0 1 1 0 0 0 1 1 1 0 0 0 1 0 1 1 0 1 0
 1 0 1 1 0 1 1 1 1 0 1 0 1 1 0 1 0 0 0 1 1 0 0 0 0 1 1 1 1 0 0 0 1 1 0 1 1
 0 1 0 1 1 1 1 0 0 0 1 1 1 1 1 0 1 0 1 0 0 0 1 1 1 1 1 0 1 1 1 0 1 1 0 0 1
 1 1 0 1 1 1 1 0 0 1 0 0 1 0 0 0 1 0 0 1 1 1 1 0 0 0 0 1 1 0 0 0 0 1 1 1 1
 1 0 1 1 1 1 1 1 0 1 1 1 1 0 0 1 1 1 0 1 0 1 0 1 1 1 1 1 0 0 1 0 0 1 1 1 1
 1 0 1 1 0 1 0 0 0 1 1 0 1 0 0 1 1 0 0 1 0 0 0 1 1 1 0 1 1 1 1 1 1 1 1 1 0
 0 0 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 0 1 1 1 0 0 1 0 1 1 1 1 1
 1 1 1 1 0 1 1 1 1 0 1 0 0 1 0 1 1 1 0 1 1 0 1 0 0 1 1 0 1 0 1 0 1 1 1 0 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 0 1 0 1 1 0 1 0 0 1 1 1 0 1 0 1
 1 1 1 0 0 1 1 0 1 0 1 0 0 1 1 1 0 1 0 1 0 1 1 1 1 1 1 1 1 0 1 0 1 1 1 0 1
 0 1 1 1 1 0 1 0 1 1 1 0 1 0 1 0 1 0 1 0 1 0 1 0 0 1 1 1 0 0 0 0 0 0 1 0 1
 0 0 1 1 0 1 0 1 1 0 0 0 1 0 1 1 0 0 1 1 1 0 0 0 1 1 1 1 1 1 0 1 1 1 1 0 1
 1 1 1 0 0 1 1 1

In [21]:
x_train_normalization = train_x / 255.0
x_test_normalization = test_x / 255.0

In [22]:
from sklearn.utils import shuffle


x_shuffled_default, y_shuffled_default = shuffle(x_train_normalization, y_train_one_hot)

In [ ]:
 inception_v4.fit(x_shuffled_default, y_shuffled_default, batch_size=256, epochs=3)

Epoch 1/3
